# Words Spelling Correction using Char2Vec

## Data Preparing

Load corpus: **PyThaiNLP** thai word corpus

In [1]:
from pythainlp.corpus import thai_words
import numpy as np

words = thai_words()
words = np.array(list(words))  # to array

In [2]:
print('Number of vocabs: ', len(words))

Number of vocabs:  62847


In [3]:
print(words[:10])  # sample

['ปรเมศวร์' 'บาดหมาย' 'สำริด' 'เสม็ดชุน' 'หมักหมม' 'อัตรชะ' 'ประหารชีวิต'
 'ประนีประนอม' 'คณะมัณฑนศิลป์' 'รักยม']


Convert array to string separate by '\n'

In [4]:
words_str = '\n'.join(words)

In [5]:
print(words_str[:100])  # sample

ปรเมศวร์
บาดหมาย
สำริด
เสม็ดชุน
หมักหมม
อัตรชะ
ประหารชีวิต
ประนีประนอม
คณะมัณฑนศิลป์
รักยม
ความรู้สึ


Convert string to char

In [6]:
words_char = list(words_str)

In [7]:
print(words_char[:100])  # sample

['ป', 'ร', 'เ', 'ม', 'ศ', 'ว', 'ร', '์', '\n', 'บ', 'า', 'ด', 'ห', 'ม', 'า', 'ย', '\n', 'ส', 'ำ', 'ร', 'ิ', 'ด', '\n', 'เ', 'ส', 'ม', '็', 'ด', 'ช', 'ุ', 'น', '\n', 'ห', 'ม', 'ั', 'ก', 'ห', 'ม', 'ม', '\n', 'อ', 'ั', 'ต', 'ร', 'ช', 'ะ', '\n', 'ป', 'ร', 'ะ', 'ห', 'า', 'ร', 'ช', 'ี', 'ว', 'ิ', 'ต', '\n', 'ป', 'ร', 'ะ', 'น', 'ี', 'ป', 'ร', 'ะ', 'น', 'อ', 'ม', '\n', 'ค', 'ณ', 'ะ', 'ม', 'ั', 'ณ', 'ฑ', 'น', 'ศ', 'ิ', 'ล', 'ป', '์', '\n', 'ร', 'ั', 'ก', 'ย', 'ม', '\n', 'ค', 'ว', 'า', 'ม', 'ร', 'ู', '้', 'ส', 'ึ']


Save data to **words-char.txt** file

In [8]:
with open('words-char.txt', mode='w', encoding='utf-8') as file:
    file.write(' '.join(words_char))

## Char2Vec Model
Using FastText traing Word2Vec model on **character level**

In [9]:
import fasttext

Load file and model training

In [10]:
model = fasttext.train_unsupervised('words-char.txt', epoch=200, ws=3)

In [11]:
print('Number of char: ', len(model.words))

Number of char:  73


Create word vector on character level model

In [12]:
words_vec = [model.get_sentence_vector(' '.join(list(word))) for word in words]
words_vec = np.array(words_vec)

## Word Spelling Correction
Using **nearest neighbors model** to find **word similarity** for get word suggestion

Model training
- X: word vector
- y: word/vocab

In [13]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

X, y = words_vec, words
nbrs = NearestNeighbors().fit(X, y)

### Save All Model

In [14]:
import joblib

model.save_model('char2vec.bin')  # fasttext model
joblib.dump(words, 'words.joblib')
joblib.dump(nbrs, 'nbrs.joblib');

# Usage

In [15]:
import fasttext
import joblib

model = fasttext.load_model('char2vec.bin')
words = joblib.load('words.joblib')
nbrs = joblib.load('nbrs.joblib')

In [16]:
words_input = ['การบด้าน', 'สวัดี', 'vออกเลอร์', 'ปละเทศไทยย', 'อรอย']

In [17]:
word_input_vec = [model.get_sentence_vector(' '.join(list(word))) for word in words_input]
indices = nbrs.kneighbors(word_input_vec, 5, False)  # n_neighbors is 5
suggestion = words[indices]

for w, s in zip(words_input, suggestion):
    print(f'{w} \n---> {s}')

การบด้าน 
---> ['การบ้าน' 'การอาบน้ำ' 'ทำการบ้าน' 'การกวาดล้าง' 'กลับด้าน']
สวัดี 
---> ['สวัสดี' 'วสวัดดี' 'วิดัสดี' 'ดบัสวี' 'วนัสบดี']
vออกเลอร์ 
---> ['ล็อคเกอร์' 'ล็อกเกอร์' 'เดอะรอยัลกอล์ฟ' 'โอเรกอน' 'ร็อกกี้เฟลเลอร์']
ปละเทศไทยย 
---> ['ประเทศไทย' 'ไปรษณีย์ลงทะเบียน' 'การปิโตรเลียมแห่งประเทศไทย' 'ลายเทศ'
 'สถานีวิทยุกระจายเสียงแห่งประเทศไทย']
อรอย 
---> ['รอย' 'รอคอย' 'อร่อย' 'รอยต่อ' 'ออยเลอร์']


In [18]:
!jupyter nbconvert --to markdown char2vec_spelling_correction.ipynb

[NbConvertApp] Converting notebook char2vec_spelling_correction.ipynb to markdown
[NbConvertApp] Writing 3777 bytes to char2vec_spelling_correction.md
